## Upotukset

Edellisessä esimerkissä käsittelimme korkeulotteisia bag-of-words-vektoreita, joiden pituus oli `vocab_size`, ja muunsimme matalalitteiset sijaintiesitykset eksplisiittisesti harvoiksi yksi-ykkösesityksiksi. Tämä yksi-ykkösesitys ei ole muistin kannalta tehokas. Lisäksi jokaista sanaa käsitellään toisistaan riippumattomana, joten yksi-ykkösenkoodatut vektorit eivät ilmaise sanojen semanttisia samankaltaisuuksia.

Tässä osiossa jatkamme **News AG** -aineiston tutkimista. Aloitetaan lataamalla data ja hakemalla joitakin määritelmiä edellisestä osasta.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Mikä on upotus?

**Upotuksen** idea on edustaa sanoja matalampidimensionaalisilla tiheillä vektoreilla, jotka heijastavat sanan semanttista merkitystä. Myöhemmin käsittelemme, miten rakentaa merkityksellisiä sanaupotuksia, mutta toistaiseksi voimme ajatella upotuksia tapana vähentää sanavektorin dimensioita.

Upotuskerros ottaa sanan syötteenä ja tuottaa ulostulovektorin, jonka koko on määritelty `embedding_size`. Tietyssä mielessä se on hyvin samanlainen kuin `Dense`-kerros, mutta sen sijaan, että se ottaisi syötteenä yksi-kuuma-koodatun vektorin, se pystyy ottamaan sanan numeron.

Kun käytämme upotuskerrosta verkkomme ensimmäisenä kerroksena, voimme siirtyä sanojen pussi -mallista **upotuspussi**-malliin, jossa ensin muutamme tekstimme jokaisen sanan vastaavaksi upotukseksi ja sitten laskemme jonkin aggregaattifunktion näiden upotusten yli, kuten `sum`, `average` tai `max`.

![Kuva, joka näyttää upotusluokittelijan viidelle sekvenssisanalle.](../../../../../translated_images/fi/embedding-classifier-example.b77f021a7ee67eee.webp)

Luokittelijaneuroverkkomme koostuu seuraavista kerroksista:

* `TextVectorization`-kerros, joka ottaa syötteenä merkkijonon ja tuottaa tensorin token-numeroista. Määrittelemme kohtuullisen sanaston koon `vocab_size` ja jätämme vähemmän käytetyt sanat huomiotta. Syötteen muoto on 1, ja ulostulon muoto on $n$, koska saamme tulokseksi $n$ tokenia, joista jokainen sisältää numeroita välillä 0–`vocab_size`.
* `Embedding`-kerros, joka ottaa $n$ numeroa ja pienentää jokaisen numeron tiheäksi vektoriksi, jonka pituus on määritelty (esimerkissämme 100). Näin ollen syötetensorin muoto $n$ muuttuu $n\times 100$ tensoriksi.
* Aggregointikerros, joka laskee tämän tensorin keskiarvon ensimmäisen akselin yli, eli se laskee kaikkien $n$ syötetensorien keskiarvon, jotka vastaavat eri sanoja. Toteutamme tämän kerroksen käyttämällä `Lambda`-kerrosta ja välitämme siihen funktion keskiarvon laskemiseksi. Ulostulon muoto on 100, ja se on koko syötesekvenssin numeerinen esitys.
* Lopullinen `Dense` lineaarinen luokittelija.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary`-tulosteessa, **output shape** -sarakkeessa, ensimmäinen tensorin dimensio `None` vastaa minibatchin kokoa, ja toinen vastaa token-sekvenssin pituutta. Kaikilla minibatchin token-sekvensseillä on eri pituudet. Keskustelemme, miten käsitellä tätä seuraavassa osiossa.

Nyt harjoitellaan verkkoa:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Huomaa** että rakennamme vektoroijaa tietojoukon osajoukon perusteella. Tämä tehdään prosessin nopeuttamiseksi, ja se saattaa johtaa tilanteeseen, jossa kaikkia tekstimme tokenoita ei ole sanastossa. Tällöin nämä tokenit jätetään huomiotta, mikä voi johtaa hieman alhaisempaan tarkkuuteen. Kuitenkin todellisessa elämässä tekstin osajoukko antaa usein hyvän arvion sanastosta.


### Käsitellään muuttuvia sekvenssikokoja

Ymmärretään, miten koulutus tapahtuu pienissä erissä. Yllä olevassa esimerkissä syöte-tenzorin ulottuvuus on 1, ja käytämme 128:n kokoisia pieneriä, jolloin tensorin todellinen koko on $128 \times 1$. Kuitenkin jokaisessa lauseessa olevien tokenien määrä vaihtelee. Jos sovellamme `TextVectorization`-kerrosta yhteen syötteeseen, palautettujen tokenien määrä vaihtelee sen mukaan, miten teksti on tokenisoitu:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Kuitenkin, kun sovellamme vektoroijaa useisiin sekvensseihin, sen täytyy tuottaa suorakulmainen tensorimuoto, joten se täyttää käyttämättömät elementit PAD-tokenilla (joka meidän tapauksessamme on nolla):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Tässä näemme upotukset:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Huom**: Jotta täydennystä voidaan minimoida, joissakin tapauksissa on järkevää järjestää kaikki tietojoukon sekvenssit pituuden mukaan nousevaan järjestykseen (tai tarkemmin sanottuna tokenien lukumäärän mukaan). Tämä varmistaa, että kukin minibatch sisältää samankaltaisen pituisia sekvenssejä.


## Semanttiset upotukset: Word2Vec

Edellisessä esimerkissämme upotuskerros oppi kartoittamaan sanat vektoriesityksiksi, mutta näillä esityksillä ei ollut semanttista merkitystä. Olisi hyödyllistä oppia vektoriesitys siten, että samankaltaiset sanat tai synonyymit vastaavat vektoreita, jotka ovat lähellä toisiaan jonkin vektorietäisyyden (esimerkiksi euklidisen etäisyyden) perusteella.

Tämän saavuttamiseksi meidän täytyy esikouluttaa upotusmallimme suurella tekstikokoelmalla käyttämällä tekniikkaa, kuten [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Se perustuu kahteen pääarkkitehtuuriin, joita käytetään sanojen hajautettujen esitysten tuottamiseen:

 - **Jatkuva sanapussimalli** (CBoW), jossa mallia koulutetaan ennustamaan sana ympäröivän kontekstin perusteella. Annettuna ngrammi $(W_{-2},W_{-1},W_0,W_1,W_2)$, mallin tavoitteena on ennustaa $W_0$ käyttäen $(W_{-2},W_{-1},W_1,W_2)$.
 - **Jatkuva skip-gram** on CBoW:n vastakohta. Malli käyttää ympäröivää kontekstisanan ikkunaa ennustaakseen nykyisen sanan.

CBoW on nopeampi, kun taas skip-gram on hitaampi, mutta se edustaa harvinaisia sanoja paremmin.

![Kuva, joka näyttää sekä CBoW- että Skip-Gram-algoritmit sanojen muuntamiseksi vektoreiksi.](../../../../../translated_images/fi/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Kokeillaksemme Word2Vec-upotusta, joka on esikoulutettu Google News -aineistolla, voimme käyttää **gensim**-kirjastoa. Alla etsimme sanoja, jotka ovat lähimpänä sanaa 'neural'.

> **Huom:** Kun luot sanavektoreita ensimmäistä kertaa, niiden lataaminen voi kestää jonkin aikaa!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Voimme myös poimia sanasta vektoriesityksen, jota voidaan käyttää luokittelumallin kouluttamisessa. Vektoriesityksessä on 300 komponenttia, mutta tässä näytämme selkeyden vuoksi vain vektorin ensimmäiset 20 komponenttia:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Hienoa semanttisissa upotuksissa on se, että voit manipuloida vektorisalausta semantiikan perusteella. Esimerkiksi voimme pyytää löytämään sanan, jonka vektoriedustus on mahdollisimman lähellä sanoja *kuningas* ja *nainen*, ja mahdollisimman kaukana sanasta *mies*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Esimerkki yllä käyttää sisäistä GenSym-taikuutta, mutta taustalla oleva logiikka on itse asiassa melko yksinkertainen. Mielenkiintoinen asia upotuksissa on, että voit suorittaa normaaleja vektorioperaatioita upotusvektoreilla, ja tämä heijastaa operaatioita sanojen **merkityksissä**. Esimerkki yllä voidaan ilmaista vektorioperaatioiden avulla: laskemme vektorin, joka vastaa **KING-MAN+WOMAN** (operaatiot `+` ja `-` suoritetaan vastaavien sanojen vektoriedustuksilla), ja sitten etsimme sanakirjasta lähimmän sanan kyseiseen vektoriin:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Jouduimme lisäämään pienet kertoimet *man*- ja *woman*-vektoreihin – kokeile poistaa ne ja katso, mitä tapahtuu.

Lähimmän vektorin löytämiseksi käytämme TensorFlow-työkaluja laskemaan etäisyysvektorin oman vektorimme ja sanaston kaikkien vektorien välillä, ja sitten löydämme pienimmän sanan indeksin käyttämällä `argmin`.


Vaikka Word2Vec vaikuttaa hyvältä tavalta ilmaista sanojen semantiikkaa, sillä on monia haittoja, mukaan lukien seuraavat:

* Sekä CBoW- että skip-gram-mallit ovat **ennustavia upotuksia**, ja ne ottavat huomioon vain paikallisen kontekstin. Word2Vec ei hyödynnä globaalia kontekstia.
* Word2Vec ei ota huomioon sanojen **morfologiaa**, eli sitä, että sanan merkitys voi riippua sanan eri osista, kuten juuresta.

**FastText** pyrkii voittamaan toisen rajoituksen ja rakentaa Word2Vecin pohjalta oppimalla vektoriedustuksia jokaiselle sanalle ja kunkin sanan sisältämille merkkien n-grammeille. Näiden edustusten arvot keskiarvotetaan yhdeksi vektoriksi jokaisessa harjoitusvaiheessa. Vaikka tämä lisää paljon lisälaskentaa esikoulutukseen, se mahdollistaa sana-upotusten koodaavan osasanatietoa.

Toinen menetelmä, **GloVe**, käyttää erilaista lähestymistapaa sana-upotuksiin, perustuen sana-konteksti-matriisin faktorisointiin. Ensin se rakentaa suuren matriisin, joka laskee sanojen esiintymiskerrat eri konteksteissa, ja sitten se yrittää esittää tämän matriisin pienemmissä ulottuvuuksissa tavalla, joka minimoi rekonstruointitappion.

Gensim-kirjasto tukee näitä sana-upotuksia, ja voit kokeilla niitä muuttamalla yllä olevaa mallin latauskoodia.


## Esikoulutettujen upotusten käyttö Kerasissa

Voimme muokata yllä olevaa esimerkkiä esitäyttääksemme upotuskerroksemme matriisin semanttisilla upotuksilla, kuten Word2Vec. Esikoulutetun upotuksen ja tekstikorpuksen sanastot eivät todennäköisesti vastaa toisiaan, joten meidän on valittava yksi. Tässä tutkimme kahta mahdollista vaihtoehtoa: tokenisoijan sanaston käyttöä ja Word2Vec-upotusten sanaston käyttöä.

### Tokenisoijan sanaston käyttö

Kun käytämme tokenisoijan sanastoa, osalla sanaston sanoista on vastaavat Word2Vec-upotukset, mutta osa puuttuu. Koska sanastomme koko on `vocab_size` ja Word2Vec-upotuksen vektoripituus on `embed_size`, upotuskerros esitetään painomatriisina, jonka muoto on `vocab_size`$\times$`embed_size`. Täytämme tämän matriisin käymällä sanaston läpi:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Sanat, joita ei löydy Word2Vec-sanakirjasta, voidaan joko jättää nolliksi tai luoda niille satunnaisvektori.

Nyt voimme määritellä upotuskerroksen esikoulutetuilla painoilla:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Huomio**: Huomaa, että asetamme `trainable=False` luodessamme `Embedding`-kerroksen, mikä tarkoittaa, että emme uudelleenkouluta Embedding-kerrosta. Tämä saattaa hieman heikentää tarkkuutta, mutta nopeuttaa koulutusta.

### Embedding-sanakirjan käyttö

Yksi ongelma aiemmassa lähestymistavassa on, että TextVectorization- ja Embedding-kerrosten käyttämät sanakirjat ovat erilaisia. Tämän ongelman ratkaisemiseksi voimme käyttää jotakin seuraavista ratkaisuista:
* Uudelleenkouluta Word2Vec-malli käyttämällä omaa sanakirjaamme.
* Lataa datasetti käyttämällä Word2Vec-mallin esikoulutettua sanakirjaa. Datasetin lataamiseen käytettävät sanakirjat voidaan määrittää latauksen yhteydessä.

Jälkimmäinen lähestymistapa vaikuttaa helpommalta, joten toteutetaan se. Ensimmäiseksi luomme `TextVectorization`-kerroksen määritetyllä sanakirjalla, joka on otettu Word2Vec-embeddingeistä:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensim-sanan upotuskirjasto sisältää kätevän funktion, `get_keras_embeddings`, joka luo automaattisesti vastaavan Keras-upotuskerroksen sinulle.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Yksi syy siihen, miksi emme näe korkeampaa tarkkuutta, on se, että jotkut sanamme datasta puuttuvat esikoulutetusta GloVe-sanakirjasta, ja siksi ne käytännössä jätetään huomiotta. Tämän voittamiseksi voimme kouluttaa omat upotuksemme perustuen omaan dataamme.


## Kontekstuaaliset upotukset

Yksi perinteisten esikoulutettujen upotusten, kuten Word2Vecin, keskeisistä rajoituksista on se, että vaikka ne voivat vangita jonkin verran sanan merkitystä, ne eivät pysty erottamaan eri merkityksiä toisistaan. Tämä voi aiheuttaa ongelmia jatkomalleissa.

Esimerkiksi sana 'play' tarkoittaa eri asioita näissä kahdessa lauseessa:
- Kävin teatterissa katsomassa **näytelmän**.
- John haluaa **leikkiä** ystäviensä kanssa.

Esikoulutetut upotukset, joista puhuimme, edustavat molempia sanan 'play' merkityksiä samalla upotuksella. Tämän rajoituksen voittamiseksi meidän täytyy rakentaa upotuksia, jotka perustuvat **kielimalliin**, joka on koulutettu suurella tekstikorpuksella ja *ymmärtää*, miten sanoja voidaan yhdistää eri konteksteissa. Kontekstuaalisten upotusten käsittely on tämän tutoriaalin ulkopuolella, mutta palaamme niihin, kun puhumme kielimalleista seuraavassa osiossa.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
